<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/LAB03_Pr4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importación de librerías a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from numpy.core.function_base import linspace
from math import floor
from math import ceil
from numpy.ma.core import size

from sklearn import tree
from sklearn.datasets import load_iris
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [3]:
# Lectura de la base de datos
filename = 'lluvias.csv'
data = pd.read_csv(filename, sep = ',')

# Revisión de los datos
print(data.shape)

data.info()
data.head(10)

(25551, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25551 entries, 0 to 25550
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    25551 non-null  object 
 1   PRCP    25548 non-null  float64
 2   TMAX    25551 non-null  int64  
 3   TMIN    25551 non-null  int64  
 4   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 998.2+ KB


,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
5,1948-01-06,0.44,48,39,True
6,1948-01-07,0.41,50,40,True
7,1948-01-08,0.04,48,35,True
8,1948-01-09,0.12,50,31,True
9,1948-01-10,0.74,43,34,True


In [4]:
# Elimino los na pq solo hay 3.
data = data.dropna()

# Quitamos los duplicados
data = data.drop_duplicates()

# data.info()

In [5]:
temp_max = data["TMAX"].max()

print('La temperatura máxima encontrada es', temp_max,'entendemos que estamos en Fº y queremos trabajar en Cº.')

La temperatura máxima encontrada es 103 entendemos que estamos en Fº y queremos trabajar en Cº.


In [6]:
data["TMIN"] = (data.TMIN - 32)*(5/9)

data["TMAX"] = (data.TMAX - 32)*(5/9)

temp_max = data["TMAX"].max().round(2)

temp_min = data["TMIN"].min().round(2)

print('La temperatura máxima en grados Celsius es', temp_max,'Cº.')

La temperatura máxima en grados Celsius es 39.44 Cº.


In [7]:
data.head(40)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,10.555556,5.555556,True
1,1948-01-02,0.59,7.222222,2.222222,True
2,1948-01-03,0.42,7.222222,1.666667,True
3,1948-01-04,0.31,7.222222,1.111111,True
4,1948-01-05,0.17,7.222222,0.000000,True
5,1948-01-06,0.44,8.888889,3.888889,True
6,1948-01-07,0.41,10.000000,4.444444,True
7,1948-01-08,0.04,8.888889,1.666667,True
8,1948-01-09,0.12,10.000000,-0.555556,True
9,1948-01-10,0.74,6.111111,1.111111,True


In [8]:
conditions = [
          (data['TMIN'] < 10),
          ((data['TMIN'] >= 10) & (data['TMIN'] <= 20)),
          (data['TMIN'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMIN'] = np.select(conditions, results)

In [9]:
conditions = [
          (data['TMAX'] < 10),
          ((data['TMAX'] >= 10) & (data['TMAX'] <= 20)),
          (data['TMAX'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMAX'] = np.select(conditions, results)

In [10]:
# data.value_counts(data['TMIN'])

data.value_counts(data['TMAX'])

TMAX
Media    12856
Alta      6618
Baja      6074
dtype: int64

In [11]:
conditions = [
          (data['PRCP'] < 1),
          ((data['PRCP'] >= 1) & (data['PRCP'] <= 2)),
          (data['PRCP'] > 2)
             ]

#define results
results = ['Poco', 'Normal', 'Mucho']

#create new column based on conditions in column1 and column2
data['PRCP'] = np.select(conditions, results)

data.head(20)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,Poco,Media,Baja,True
1,1948-01-02,Poco,Baja,Baja,True
2,1948-01-03,Poco,Baja,Baja,True
3,1948-01-04,Poco,Baja,Baja,True
4,1948-01-05,Poco,Baja,Baja,True
5,1948-01-06,Poco,Baja,Baja,True
6,1948-01-07,Poco,Media,Baja,True
7,1948-01-08,Poco,Baja,Baja,True
8,1948-01-09,Poco,Media,Baja,True
9,1948-01-10,Poco,Baja,Baja,True


In [163]:
data.value_counts(data['PRCP'])
# data['TMAX'].value_counts()

PRCP
Poco      25189
Normal      327
Mucho        32
dtype: int64

Vamos a contar cada una de las distribuciones de clases para los diferentes atributs.

TODO: Hay que ver como clasificar en función de si es True or False (NO a mano)

In [162]:
# Guardamos la frecuencia del valor 'True' y 'False' en variables
total_rain_true  = data['RAIN'].value_counts()[True]
total_rain_false = data['RAIN'].value_counts()[False]

# Total de registros
total_registros = total_rain_true + total_rain_false

# P(True)
prob_true = total_rain_true/total_registros

data_rain_true = data.query('RAIN != @False')

# data_rain_true.head(40)
# prob_true
total_rain_true
# data_rain_false = data.query('RAIN != @True')

10900

In [161]:
# Contamos la frecuencia de los valores en la columna de clase 'PRCP' con 'RAIN' == 'True'

total_p_PRCP = data_rain_true['PRCP'].value_counts()['Poco']
total_n_PRCP = data_rain_true['PRCP'].value_counts()['Normal']
total_m_PRCP = data_rain_true['PRCP'].value_counts()['Mucho']

print(total_m_PRCP)

32


In [129]:
# Guardamos la frecuencia de los valores 'Baja', 'Media' y 'Alta' en variables con 'RAIN' == 'True'
total_b_TMAX = data_rain_true['TMAX'].value_counts()['Baja']
total_m_TMAX = data_rain_true['TMAX'].value_counts()['Media']
total_a_TMAX = data_rain_true['TMAX'].value_counts()['Alta']

print(total_a_TMAX)

621


In [130]:
# Contamos la frecuencia de los valores en la columna de clase 'TMAX' con 'RAIN' == 'True'
total_b_TMIN = data_rain_true['TMIN'].value_counts()['Baja']
total_m_TMIN = data_rain_true['TMIN'].value_counts()['Media']
total_a_TMIN = 0

# total_a_TMIN = data_rain_true['TMIN'].value_counts()['Alta']  
# TODO NO HAY NINGUNA

print(total_m_TMIN)

2283


In [159]:
# Creamos un dataframe con los nuevos registros
data_test = pd.DataFrame({'Date': ['15-12-2017', '16-12-2017', '17-12-2017'], 'PRCP': ['Poco', 'Mucho', 'Normal'], 'TMAX': ['Media', 'Baja', 'Alta'], 'TMIN': ['Baja', 'Media', 'Baja']})

In [160]:
# Buscamos los valores de los atributos para poder realizar la clasificación basada en redes bayesianas

# Iteramos sobre las filas del dataframe nuevo con un bucle for
for indice, fila in data_test.iterrows():
    print('\nProbabilidades del día', fila['Date'],'\n')
    
    # Probabilidades de 'RAIN == True' en función de las PRCP.
    if data_test.loc[data_test.index == indice, 'PRCP'].apply(lambda x: 'Poco' in x).item():
      probabilidad_PRCP = total_p_PRCP/total_rain_true
    elif data_test.loc[data_test.index == indice, 'PRCP'].apply(lambda x: 'Normal' in x).item():
      probabilidad_PRCP = total_n_PRCP/total_rain_true
    else:
      probabilidad_PRCP = total_m_PRCP/total_rain_true
    print(probabilidad_PRCP)

    # Probabilidades de 'RAIN == True' en función de la TMAX.
    if data_test.loc[data_test.index == indice, 'TMAX'].apply(lambda x: 'Baja' in x).item():
      probabilidad_TMAX = total_b_TMAX/total_rain_true
    elif data_test.loc[data_test.index == indice, 'TMAX'].apply(lambda x: 'Media' in x).item():
      probabilidad_TMAX = total_m_TMAX/total_rain_true
    else:
      probabilidad_TMAX = total_a_TMAX/total_rain_true
    print(probabilidad_TMAX)

    # Probabilidades de 'RAIN == True' en función de la TMIN.
    if data_test.loc[data_test.index == indice, 'TMIN'].apply(lambda x: 'Baja' in x).item():
      probabilidad_TMIN = total_b_TMIN/total_rain_true
    elif data_test.loc[data_test.index == indice, 'TMIN'].apply(lambda x: 'Media' in x).item():
      probabilidad_TMIN = total_m_TMIN/total_rain_true
    else:
      probabilidad_TMIN = total_a_TMIN/total_rain_true
    print(probabilidad_TMIN)
    print('\nLa probabilidad de lluvia del día', fila['Date'],' es de', probabilidad_PRCP*probabilidad_TMAX*probabilidad_TMIN*prob_true,'\n')


Probabilidades del día 15-12-2017 

0.9670642201834863
0.6200917431192661
0.7905504587155964

La probabilidad de lluvia del día 15-12-2017  es de 0.20226020792085314 


Probabilidades del día 16-12-2017 

0.0029357798165137615
0.3229357798165138
0.20944954128440366

La probabilidad de lluvia del día 16-12-2017  es de 8.47205272545984e-05 


Probabilidades del día 17-12-2017 

0.03
0.056972477064220185
0.7905504587155964

La probabilidad de lluvia del día 17-12-2017  es de 0.0005764817224781415 

